# Fully distributed execution with toksearch_submit (saga-only)


The mechanism for distributing TokSearch computations is to invoke a TokSearch script using the ```toksearch_submit``` utility. Behind the scenes, ```toksearch_submit``` uses SLURM, and can be run in both interactive and batch modes, utilizing salloc and sbatch respectively. The default mode is to run interactively, using salloc.

This will be demonstrated with an example script, adapted from the example above, and available in the TokSearch installation:

In [1]:
%%bash
cat $(which toksearch_example.py)

#!/opt/toksearch/users/sammuli/toksearch_dev/toksearch/envs/toksearch_dev/bin/python3.7
# EASY-INSTALL-DEV-SCRIPT: 'toksearch==0.0.0','toksearch_example.py'
__requires__ = 'toksearch==0.0.0'
__import__('pkg_resources').require('toksearch==0.0.0')
__file__ = '/opt/toksearch/users/sammuli/toksearch_dev/toksearch/scripts/toksearch_example.py'
with open(__file__) as f:
    exec(compile(f.read(), __file__, 'exec'))


This script can be executed using either the ```compute_ray``` or ```compute_spark``` methods. First let's see how it works using Ray:

In [2]:
%%bash
# Assume that this is being run on the saga login node and that you've done "module load toksearch"
#toksearch_example.py ray

# We'll run using 3 worker nodes. As of this writing there are 6 worker nodes available.
toksearch_submit -N 3 toksearch_example.py -- ray

Initializing ray with _temp_dir = /tmp/tmp3p8sp40m
['10.0.0.43', '10.0.0.44', '10.0.0.45']
STARTING CLUSTER
--nodes=1 --ntasks=1 -w saga03 --gres=gpu:volta:1 --overlap --propagate ray start --block --node-ip-address 10.0.0.43 --temp-dir /tmp/tmp3p8sp40m --port 6543 --head --object-store-memory=95000000000 --memory=80000000000
Ok, started head node
dict_keys(['saga04', 'saga05'])
Starting saga04...
--nodes=1 --ntasks=1 -w saga04 --gres=gpu:volta:1 --overlap --propagate ray start --block --node-ip-address 10.0.0.44 --temp-dir /tmp/tmp3p8sp40m --address=10.0.0.43:6543 --object-store-memory=95000000000 --memory=80000000000
Starting saga05...
--nodes=1 --ntasks=1 -w saga05 --gres=gpu:volta:1 --overlap --propagate ray start --block --node-ip-address 10.0.0.45 --temp-dir /tmp/tmp3p8sp40m --address=10.0.0.43:6543 --object-store-memory=95000000000 --memory=80000000000
********************************************************************************
BATCH 1/1
NUM CPUS: 96
NUM PARTITIONS: 10000
ME

salloc: Granted job allocation 15924
salloc: Waiting for resource configuration
salloc: Nodes saga[03-05] are ready for job
srun: error: ioctl(TIOCGWINSZ): Inappropriate ioctl for device
srun: error: Not using a pseudo-terminal, disregarding --pty option
salloc: Relinquishing job allocation 15924


Similarly, we can run with Spark:

In [3]:
%%bash

toksearch_submit -N 3 toksearch_example.py -- spark

['10.0.0.43', '10.0.0.44', '10.0.0.45']
STARTING CLUSTER
MASTER IP 10.0.0.43
['--nodes=1', '--ntasks=1', '-w', 'saga03', '--gres=gpu:volta:0', '--mem=150G', '/opt/toksearch/deps/spark/sbin/start-master.sh']
Ok, started head node
dict_keys(['saga03', 'saga04', 'saga05'])
Starting saga03...
['--nodes=1', '--ntasks=1', '-w', 'saga03', '--gres=gpu:volta:1', '--mem=150G', '/opt/toksearch/deps/spark/sbin/start-slave.sh', 'spark://10.0.0.43:7077', '-i', '10.0.0.43', '-c', 48, '-m', '149G']
Starting saga04...
['--nodes=1', '--ntasks=1', '-w', 'saga04', '--gres=gpu:volta:1', '--mem=150G', '/opt/toksearch/deps/spark/sbin/start-slave.sh', 'spark://10.0.0.43:7077', '-i', '10.0.0.44', '-c', 48, '-m', '149G']
Starting saga05...
['--nodes=1', '--ntasks=1', '-w', 'saga05', '--gres=gpu:volta:1', '--mem=150G', '/opt/toksearch/deps/spark/sbin/start-slave.sh', 'spark://10.0.0.43:7077', '-i', '10.0.0.45', '-c', 48, '-m', '149G']
Got 10000 results using spark


salloc: Granted job allocation 7454
srun: error: ioctl(TIOCGWINSZ): Inappropriate ioctl for device
srun: error: Not using a pseudo-terminal, disregarding --pty option
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/03/24 08:34:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
salloc: Relinquishing job allocation 7454
salloc: Job allocation 7454 has been revoked.


## ```toksearch_submit``` syntax

Now that we've seen a few examples, let's describe in a bit more detail what's actually going on. Taking one of the examples again:

```bash
toksearch__submit -N 3 toksearch_example.py -- spark
```

Let's break down the syntax:

- ```-N 3```: This specifies the use of 3 worker nodes. As of this writing ```toksearch_submit``` only supports using entire nodes (and not a subset of cores within a node)

- ```toksearch_example.py```: This is the script we want to run. We can alternatively invoke this as follows:
    - ```toksearch_submit -N 3 python -- toksearch_example.py spark```
    
- ``` -- ```: The double-dash between the script and subsequent arguments is used to denote that everything following it are arguments that belong to the script, and not ```toksearch_submit```. So in this case ```spark``` is an argument to ```toksearch_example.py``` (and not an argument to ```toksearch_submit```).

## Getting help

Help for ```toksearch_submit```, with a full listing of all options is available using the ```--help``` option:

In [4]:
%%bash

toksearch_submit --help

usage: toksearch_submit [-h] [-N NUM_NODES] [-b] [--config-file CONFIG_FILE]
                        script [script_args [script_args ...]]

positional arguments:
  script                User-provided script
  script_args           Arguments/options for user-provided script

optional arguments:
  -h, --help            show this help message and exit
  -N NUM_NODES, --num-nodes NUM_NODES
                        Number of cluster nodes to use
  -b, --batch           Run in batch mode (ie use sbatch)
  --config-file CONFIG_FILE
                        Config file used to set up slurm
